In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import catboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmc-ml-spotify-recommendations/sample_submission.csv
/kaggle/input/cmc-ml-spotify-recommendations/added_info.csv
/kaggle/input/cmc-ml-spotify-recommendations/audio_features.csv
/kaggle/input/cmc-ml-spotify-recommendations/simple_pipeline.ipynb
/kaggle/input/cmc-ml-spotify-recommendations/tracks_info.csv
/kaggle/input/cmc-ml-spotify-recommendations/train.csv
/kaggle/input/cmc-ml-spotify-recommendations/test.csv
/kaggle/input/spotify-charts-all-audio-data/merged_data.csv
/kaggle/input/spotify-1-million-tracks-3-feature-dataset/train_ext.csv
/kaggle/input/spotify-1-million-tracks-3-feature-dataset/test_ext.csv
/kaggle/input/spotify-1-3-million-tracks-10-features/train_album_info.csv
/kaggle/input/spotify-1-3-million-tracks-10-features/test_album_info.csv


<h3> This notebook uses 3 datasets: 2 public and 1 private <br>
    The private dataset contains information about track genres<br>
The other two may be accessed via "Input" folder :)</h3>

In [2]:
# Используйте эти пути для запуска ноутбука на Kaggle
PATH_TO_KAGGLE_TRAIN = "/kaggle/input/cmc-ml-spotify-recommendations/train.csv"
PATH_TO_KAGGLE_TEST = "/kaggle/input/cmc-ml-spotify-recommendations/test.csv"
PATH_TO_KAGGLE_SUBMISSION = "/kaggle/working/submission.csv"

# Если запускаете на своем компьютере, то впишите актуальные пути
# PATH_TO_TRAIN = # Your Path
# PATH_TO_TEST = # Your Path
# PATH_TO_SUBMISSION = # Your Path

train = pd.read_csv(PATH_TO_KAGGLE_TRAIN)
test = pd.read_csv(PATH_TO_KAGGLE_TEST)

X_train = train.drop(["index", "exists"], axis=1)
y_train = train["exists"]

train.head()

,index,playlist_id,track_id,exists
0,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1
1,645550,35527,1PEqh7awkpuepLBSq8ZwqD,1
2,2630414,12203,7E8pPgBY84oDaXRcqODavR,0
3,3168314,21850,0Atml4huw4Fgyk6YSHiK4M,0
4,3144464,21391,4WYDmIZrwxBHdBYdvi5oQO,0


In [3]:
added_info = '/kaggle/input/cmc-ml-spotify-recommendations/added_info.csv'
added_info = pd.read_csv(added_info)

all_info_path = '/kaggle/input/spotify-charts-all-audio-data/merged_data.csv'

add_3_train = '/kaggle/input/spotify-1-million-tracks-3-feature-dataset/train_ext.csv'
add_3_test = '/kaggle/input/spotify-1-million-tracks-3-feature-dataset/test_ext.csv'

add_10_train = '/kaggle/input/spotify-1-3-million-tracks-10-features/train_album_info.csv'
add_10_test = '/kaggle/input/spotify-1-3-million-tracks-10-features/test_album_info.csv'

audio_features = '/kaggle/input/cmc-ml-spotify-recommendations/audio_features.csv'
audio_features = pd.read_csv(audio_features)

tracks_info_path = '/kaggle/input/cmc-ml-spotify-recommendations/tracks_info.csv'

In [4]:
partial_tracks = pd.read_csv(tracks_info_path, nrows=100)
partial_tracks

,track_id,track_artists,track_available_markets,track_disc_number,track_duration_ms,track_episode,track_explicit,track_is_local,track_name,track_popularity,...,track_album_duration_ms,track_album_episode,track_album_explicit,track_album_id,track_album_is_local,track_album_name,track_album_popularity,track_album_track,track_album_track_number,track_album_type
0,5SrPeOqm3SBecyiAsI9HTT,[{'external_urls': {'spotify': 'https://open.s...,[],1.0,140288.0,False,True,False,Perros Blancos Freestyle,0.0,...,140288.0,False,True,5SrPeOqm3SBecyiAsI9HTT,False,Perros Blancos Freestyle,0.0,True,1.0,track
1,0YfvJaebmC1ch3vMik9yuA,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,514909.0,False,False,False,Cho Ku Rei • Que Todo el Poder del Universo Es...,40.0,...,514909.0,False,False,0YfvJaebmC1ch3vMik9yuA,False,Cho Ku Rei • Que Todo el Poder del Universo Es...,40.0,True,1.0,track
2,6bSpavIeJURyGBNMUlJydq,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,254306.0,False,False,False,Wayfaring Stranger,12.0,...,254306.0,False,False,6bSpavIeJURyGBNMUlJydq,False,Wayfaring Stranger,12.0,True,15.0,track
3,0fIBz9zLZGqrCMaAUskhk2,[{'external_urls': {'spotify': 'https://open.s...,"['AE', 'AR', 'AT', 'AU', 'BA', 'BD', 'BE', 'BH...",1.0,576653.0,False,False,False,"Symphony No. 3 in A Minor, Op. 56, MWV N 18 ""S...",20.0,...,576653.0,False,False,0fIBz9zLZGqrCMaAUskhk2,False,"Symphony No. 3 in A Minor, Op. 56, MWV N 18 ""S...",20.0,True,4.0,track
4,2fG23IIy7rYN8eMNbAHcS2,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,196040.0,False,False,False,Keep It To Yourself,52.0,...,196040.0,False,False,2fG23IIy7rYN8eMNbAHcS2,False,Keep It To Yourself,52.0,True,9.0,track
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6yxYagNp6Gapd7jXmWzgVc,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,176721.0,False,False,False,A Culpa É Dele - Ao Vivo | Acústico,57.0,...,176721.0,False,False,6yxYagNp6Gapd7jXmWzgVc,False,A Culpa É Dele - Ao Vivo | Acústico,57.0,True,2.0,track
96,6ouTDaifNW4MVhKCobfZus,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,133125.0,False,True,False,Jackie Chan - Laidback Luke Remix,21.0,...,133125.0,False,True,6ouTDaifNW4MVhKCobfZus,False,Jackie Chan - Laidback Luke Remix,21.0,True,1.0,track
97,7g6NvjF49v2DyHbURBSGmD,[{'external_urls': {'spotify': 'https://open.s...,[],1.0,410800.0,False,False,False,Metal Blessing (Acoustic Live at the Seaside F...,0.0,...,410800.0,False,False,7g6NvjF49v2DyHbURBSGmD,False,Metal Blessing (Acoustic Live at the Seaside F...,0.0,True,33.0,track
98,1xyDxDZZLNsDAUgs75up9G,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AL', 'AM', 'AT', 'AU', 'AZ', 'BA', 'BE...",1.0,203226.0,False,False,False,Oasis,26.0,...,203226.0,False,False,1xyDxDZZLNsDAUgs75up9G,False,Oasis,26.0,True,4.0,track


In [5]:
partial_charts = pd.read_csv(all_info_path, nrows=100)
partial_charts

,Unnamed: 0,title,rank,date,artist,url,region,chart,trend,streams,...,af_key,af_loudness,af_mode,af_speechiness,af_acousticness,af_instrumentalness,af_liveness,af_valence,af_tempo,af_time_signature
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.907,102.034,4.0
1,1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.533,99.935,4.0
2,2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.710,93.974,4.0
3,3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.555,180.044,4.0
4,4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.873,88.007,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Dale Tu Amor (feat. El Polaco),96,2017-01-01,Agapornis,https://open.spotify.com/track/3W9HjL7lmbYXL4k...,Argentina,top200,MOVE_UP,24345.0,...,11.0,-5.522,1.0,0.0616,0.32800,0.000000,0.0567,0.922,105.981,4.0
96,96,Me Reclama,97,2017-01-01,"Mambo Kingz, DJ Luian, Luigi 21 Plus, Ozuna",https://open.spotify.com/track/4ojwGTehgBRAg52...,Argentina,top200,MOVE_DOWN,24260.0,...,1.0,-4.215,0.0,0.0826,0.34800,0.000000,0.1680,0.713,175.930,4.0
97,97,Márchate ahora,98,2017-01-01,Los Totora,https://open.spotify.com/track/3voxukrHBXctzxd...,Argentina,top200,MOVE_UP,24133.0,...,7.0,-6.579,1.0,0.0784,0.14800,0.000000,0.0886,0.754,101.946,4.0
98,98,Bronceado,99,2017-01-01,Marama,https://open.spotify.com/track/4smbzhaRt7EYwO7...,Argentina,top200,MOVE_UP,24077.0,...,0.0,-5.256,1.0,0.0550,0.20600,0.000000,0.0913,0.946,142.568,3.0


In [6]:
partial_3 = pd.read_csv(add_3_train, nrows=100)
partial_3

,track_id,genres,artist_followers,artist_popularity
0,07vS8obfeZbr8H4MgQfXR7,"['indie pop', 'la indie', 'pov: indie']",2338837.0,74
1,1PEqh7awkpuepLBSq8ZwqD,"['lilith', 'new wave pop']",84914.0,51
2,7E8pPgBY84oDaXRcqODavR,"['deep groove house', 'house', 'tech house']",59150.0,54
3,0Atml4huw4Fgyk6YSHiK4M,[],1528.0,0
4,4WYDmIZrwxBHdBYdvi5oQO,"['chill lounge', 'deep chill']",6776.0,28
...,...,...,...,...
95,0CAfXk7DXMnon4gLudAp7J,"['dance pop', 'miami hip hop', 'pop', 'pop rap']",8437254.0,76
96,4mL9OpyXvFzB315bjgxmZE,"['italian renaissance', 'medieval', 'trecento']",346.0,3
97,3PHPlTmRwOtmJglZfWkqh5,"['adult standards', 'folk', 'folk rock', 'mell...",573182.0,44
98,5TDYcGAoafLXA7KKPwbgzZ,[],139.0,6


In [7]:
partial_charts.iloc[0]

Unnamed: 0                                                             0
title                                            Chantaje (feat. Maluma)
rank                                                                   1
date                                                          2017-01-01
artist                                                           Shakira
url                    https://open.spotify.com/track/6mICuAdrwEjh6Y6...
region                                                         Argentina
chart                                                             top200
trend                                                      SAME_POSITION
streams                                                         253019.0
track_id                                          6mICuAdrwEjh6Y6lroV2Kg
album                                                          El Dorado
popularity                                                          78.0
duration_ms                                        

In [8]:
partial_10 = pd.read_csv(add_10_train, nrows=100)
partial_10

,Unnamed: 0,available_markets,duration_ms,explicit,id,is_local,name,popularity,album_name,album_release_date,album_total_tracks
0,0,[],220812,False,07vS8obfeZbr8H4MgQfXR7,False,Friday I’m In Love - Recorded at Spotify Studi...,0,Spotify Singles,2018-12-05,2
1,1,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",239960,False,1PEqh7awkpuepLBSq8ZwqD,False,I Love You Always Forever,71,Now in a Minute,1996-04-16,11
2,2,[],162771,False,7E8pPgBY84oDaXRcqODavR,False,Love Too Deep - Radio Edit,0,Love Too Deep,2014-07-07,2
3,3,[],218163,False,0Atml4huw4Fgyk6YSHiK4M,False,No Tiren Las Botellas,0,Voces Del Pueblo,2001-01-24,15
4,4,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",392855,False,4WYDmIZrwxBHdBYdvi5oQO,False,El Momento de Despertar - Blue Sky Mix,7,The Smooth Operator - Cosmopolitan Lounge Music,2014-10-03,26
...,...,...,...,...,...,...,...,...,...,...,...
95,95,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",231400,False,0CAfXk7DXMnon4gLudAp7J,False,Low (feat. T-Pain),82,Mail on Sunday,2008-03-17,14
96,96,[],301866,False,4mL9OpyXvFzB315bjgxmZE,False,"Amor, deh dimmi",0,Paolo da Firenze: Narcisso Speculando,2002-01-25,11
97,97,"['AR', 'AU', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL...",218800,False,3PHPlTmRwOtmJglZfWkqh5,False,Since I Don't Have You,38,Fate For Breakfast,1979-03-15,11
98,98,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",65770,False,5TDYcGAoafLXA7KKPwbgzZ,False,We Have a Gospel - Euphonium Multi-Track,4,Songs by Gowans & Larsson (Baritone Horn & Eup...,2022-03-25,8


In [9]:
partial_10.drop('Unnamed: 0', axis=1, inplace=True)

In [10]:
partial_10.rename(columns={'id': 'track_id'}, inplace=True)

In [11]:
partial_10

,available_markets,duration_ms,explicit,track_id,is_local,name,popularity,album_name,album_release_date,album_total_tracks
0,[],220812,False,07vS8obfeZbr8H4MgQfXR7,False,Friday I’m In Love - Recorded at Spotify Studi...,0,Spotify Singles,2018-12-05,2
1,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",239960,False,1PEqh7awkpuepLBSq8ZwqD,False,I Love You Always Forever,71,Now in a Minute,1996-04-16,11
2,[],162771,False,7E8pPgBY84oDaXRcqODavR,False,Love Too Deep - Radio Edit,0,Love Too Deep,2014-07-07,2
3,[],218163,False,0Atml4huw4Fgyk6YSHiK4M,False,No Tiren Las Botellas,0,Voces Del Pueblo,2001-01-24,15
4,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",392855,False,4WYDmIZrwxBHdBYdvi5oQO,False,El Momento de Despertar - Blue Sky Mix,7,The Smooth Operator - Cosmopolitan Lounge Music,2014-10-03,26
...,...,...,...,...,...,...,...,...,...,...
95,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",231400,False,0CAfXk7DXMnon4gLudAp7J,False,Low (feat. T-Pain),82,Mail on Sunday,2008-03-17,14
96,[],301866,False,4mL9OpyXvFzB315bjgxmZE,False,"Amor, deh dimmi",0,Paolo da Firenze: Narcisso Speculando,2002-01-25,11
97,"['AR', 'AU', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL...",218800,False,3PHPlTmRwOtmJglZfWkqh5,False,Since I Don't Have You,38,Fate For Breakfast,1979-03-15,11
98,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",65770,False,5TDYcGAoafLXA7KKPwbgzZ,False,We Have a Gospel - Euphonium Multi-Track,4,Songs by Gowans & Larsson (Baritone Horn & Eup...,2022-03-25,8


<h4>Upon inspection of the data, it turns out that all the features listed in the set in the cell below are noise, as these are copies of already existing features with the same information</h4>

In [12]:
all_cols = set(train.columns) \
    .union(set(partial_tracks.columns)
    .union(set(audio_features.columns)
    .union(set(added_info.columns)
    .union(set(partial_3.columns)
    .union(set(partial_10.columns)
    .union(set(['rank', 'region', 'chart', 'streams', 'trend'])))))))
all_cols -= {'id', 'track_album_artists', 'track_album_disc_number', 'track_album_duration_ms', 'track_album_episode',
    'track_album_explicit', 'track_album_id', 'track_album_is_local', 'track_album_name', 'track_album_popularity',
    'track_album_track', 'track_album_track_number', 'track_album_type',
    'is_local', 'track_explicit', 'track_duration_ms', 'track_available_markets'}

In [13]:
all_cols = list(all_cols)
all_cols

['artist_popularity',
 'track_disc_number',
 'album_release_date',
 'added_at',
 'region',
 'mode',
 'rank',
 'exists',
 'streams',
 'time_signature',
 'speechiness',
 'track_album_album',
 'playlist_id',
 'artist_followers',
 'track_popularity',
 'album_name',
 'track_name',
 'valence',
 'track_track',
 'energy',
 'track_id',
 'loudness',
 'chart',
 'duration_ms',
 'acousticness',
 'index',
 'popularity',
 'instrumentalness',
 'available_markets',
 'track_episode',
 'track_type',
 'track_artists',
 'key',
 'track_track_number',
 'name',
 'explicit',
 'track_is_local',
 'trend',
 'liveness',
 'added_by_type',
 'danceability',
 'added_by_id',
 'album_total_tracks',
 'tempo',
 'genres']

In [14]:
train.shape

(2773062, 4)

There are duplicates in the data. Let's drop them

In [15]:
added_info.drop_duplicates(subset=['track_id', 'playlist_id'], inplace=True)
audio_features.drop_duplicates(subset='id', inplace=True)

Now extending train and test feature matrices with the new information

In [16]:
extended_train = pd.merge(train, added_info, on=['track_id', 'playlist_id'], how='left')
extended_train = pd.merge(extended_train, audio_features, left_on='track_id', right_on='id', how='left')

extended_test = pd.merge(test, added_info, on=['track_id', 'playlist_id'], how='left')
extended_test = pd.merge(extended_test, audio_features, left_on='track_id', right_on='id', how='left')

In [17]:
add_3_train = pd.read_csv(add_3_train)
add_3_test = pd.read_csv(add_3_test)

In [18]:
train.shape

(2773062, 4)

In [19]:
add_3_train.drop_duplicates(subset='track_id', inplace=True)
add_3_test.drop_duplicates(subset='track_id', inplace=True)

In [20]:
test.shape

(1188456, 3)

In [21]:
extended_train = pd.merge(extended_train, add_3_train, on='track_id', how='left')
extended_test = pd.merge(extended_test, add_3_test, on='track_id', how='left')

In [22]:
# Those we no longer need
train = extended_train
del(extended_train)

test = extended_test
del(extended_test)

del(audio_features)
del(added_info)

The data may be too much to handle for the RAM all at once. Let's spare it by writing down the train and test files

In [23]:
header_df = pd.DataFrame(columns=all_cols)
train_file = '/kaggle/working/train_extend.csv'
header_df.to_csv(train_file, index=False)

In [24]:
add_10_train = pd.read_csv(add_10_train)
add_10_test = pd.read_csv(add_10_test)

In [25]:
add_10_train.drop('Unnamed: 0', axis=1, inplace=True)
add_10_test.drop('Unnamed: 0', axis=1, inplace=True)

In [26]:
add_10_train.rename(columns={'id': 'track_id'}, inplace=True)
add_10_test.rename(columns={'id': 'track_id'}, inplace=True)

In [27]:
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [28]:
train.iloc[0]

index                                                1050194
playlist_id                                            32931
track_id                              07vS8obfeZbr8H4MgQfXR7
exists                                                     1
added_at                                2021-07-04T11:06:43Z
added_by_id                                           2750.0
added_by_type                                           user
danceability                                           0.373
energy                                                 0.123
key                                                      7.0
loudness                                             -15.193
mode                                                     1.0
speechiness                                           0.0407
acousticness                                           0.948
instrumentalness                                         0.0
liveness                                              0.0816
valence                 

In [29]:
print(len(add_10_train), len(add_10_test))

856405 624341


In [30]:
train = pd.merge(train, add_10_train, on='track_id', how='left')
test = pd.merge(test, add_10_test, on='track_id', how='left')

In [31]:
print(len(add_10_train), len(add_10_test))

856405 624341


The file with "tracks_info" might be too big. It's a good idea to read it chunk by chunk

In [32]:
train.columns

Index(['index', 'playlist_id', 'track_id', 'exists', 'added_at', 'added_by_id',
       'added_by_type', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms_x', 'time_signature', 'genres',
       'artist_followers', 'artist_popularity', 'available_markets',
       'duration_ms_y', 'explicit', 'is_local', 'name', 'popularity',
       'album_name', 'album_release_date', 'album_total_tracks'],
      dtype='object')

In [33]:
pd.read_csv(tracks_info_path, nrows=100).iloc[0]

track_id                                               5SrPeOqm3SBecyiAsI9HTT
track_artists               [{'external_urls': {'spotify': 'https://open.s...
track_available_markets                                                    []
track_disc_number                                                         1.0
track_duration_ms                                                    140288.0
track_episode                                                           False
track_explicit                                                           True
track_is_local                                                          False
track_name                                           Perros Blancos Freestyle
track_popularity                                                          0.0
track_track                                                              True
track_track_number                                                        1.0
track_type                                                      

In [34]:
chunk_size = 2000000
chunks = pd.read_csv(all_info_path, chunksize=chunk_size)

i = 0
res = []
for chunk in chunks:
    chunk.drop_duplicates(subset='track_id', inplace=True)
    print(i)
    i += 1
    common_cols = list(set(train.columns) & set(chunk.columns))
    merged = pd.merge(train, chunk, on=common_cols, how='inner')
    train = pd.merge(train, merged, on=list(train.columns), how='left')
    del(merged)
    del(chunk)

/tmp/ipykernel_18/2108128790.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


0
1
2


/tmp/ipykernel_18/2108128790.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


3
4


/tmp/ipykernel_18/2108128790.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


...

10


/tmp/ipykernel_18/2108128790.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


11


/tmp/ipykernel_18/2108128790.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


12
13


In [35]:
chunk_size = 100000
chunks = pd.read_csv(tracks_info_path, chunksize=chunk_size)

i = 0
res = []
for chunk in chunks:
    chunk.drop_duplicates(subset='track_id', inplace=True)
    print(i)
    i += 1
    common_cols = list(set(train.columns) & set(chunk.columns))
    merged = pd.merge(train, chunk, on=common_cols, how='inner')
    train = pd.merge(train, merged, on=list(train.columns), how='left')
    del(merged)
    del(chunk)

0
1
2
3
4
5
6
7
8


Some of the data is in the form of strings / objects though they are actually lists or sets. Fortunately, Python provides a way to deal with that.

In [36]:
import ast

In [37]:
train['track_album_album'] = train['track_album_album'].apply(lambda x: ast.literal_eval(x)['album_type'] if pd.notna(x) else None)
train['track_artists'] = train['track_artists'].apply(lambda x: ast.literal_eval(x)[0]['name'] if pd.notna(x) else None)

In [38]:
train['genres'] = train['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else None)

Write the temporary file

In [39]:
train[all_cols].iloc[0]

artist_popularity                                                    74
track_disc_number                                                   1.0
album_release_date                                           2018-12-05
added_at                                           2021-07-04T11:06:43Z
region                                                              NaN
mode                                                                1.0
rank                                                                NaN
exists                                                                1
streams                                                             NaN
time_signature                                                      4.0
speechiness                                                      0.0407
track_album_album                                                single
playlist_id                                                       32931
artist_followers                                              23

In [40]:
train[all_cols].to_csv(train_file, index=False)
del(train)

Now the same with the "test"

In [41]:
all_cols.remove('exists')
header_df = pd.DataFrame(columns=all_cols)
test_file = '/kaggle/working/test_extend.csv'
header_df.to_csv(test_file, index=False)

chunk_size = 100000
chunks = pd.read_csv(tracks_info_path, chunksize=chunk_size)

i = 0
for chunk in chunks:
    chunk.drop_duplicates(subset='track_id', inplace=True)
    print(i)
    i += 1
    common_cols = list(set(test.columns) & set(chunk.columns))
    merged = pd.merge(test, chunk, on=common_cols, how='inner')
    test = pd.merge(test, merged, on=list(test.columns), how='left')
    del(chunk)
    del(merged)

chunk_size = 2000000
chunks = pd.read_csv(all_info_path, chunksize=chunk_size)
i = 0
for chunk in chunks:
    chunk.drop_duplicates(subset='track_id', inplace=True)
    print(i)
    i += 1
    common_cols = list(set(test.columns) & set(chunk.columns))
    merged = pd.merge(test, chunk, on=common_cols, how='inner')
    test = pd.merge(test, merged, on=list(test.columns), how='left')
    del(chunk)
    del(merged)

test['track_album_album'] = test['track_album_album'].apply(lambda x: ast.literal_eval(x)['album_type'] if pd.notna(x) else None)
test['track_artists'] = test['track_artists'].apply(lambda x: ast.literal_eval(x)[0]['name'] if pd.notna(x) else None)
test['genres'] = test['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else None)

test[all_cols].to_csv(test_file, index=False)
del(test)

0
1
2
3
4
5
6
7
8


/tmp/ipykernel_18/4199449655.py:23: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


0
1
2


/tmp/ipykernel_18/4199449655.py:23: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


3
4


...

10


/tmp/ipykernel_18/4199449655.py:23: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


11


/tmp/ipykernel_18/4199449655.py:23: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:


12
13


<h3>Intermediate results in the form of files with aggregated information have been yielded. <br> Now, closer to training the model:</h3>

Let's inspect the data a bit

In [42]:
X_train = pd.read_csv(train_file)
X_train

,artist_popularity,track_disc_number,album_release_date,added_at,region,mode,rank,exists,streams,time_signature,...,explicit,track_is_local,trend,liveness,added_by_type,danceability,added_by_id,album_total_tracks,tempo,genres
0,74,1.0,2018-12-05,2021-07-04T11:06:43Z,NaN,1.0,NaN,1,NaN,4.0,...,False,False,NaN,0.0816,user,0.373,2750.0,2,97.129,"['indie pop', 'la indie', 'pov: indie']"
1,51,NaN,1996-04-16,2023-01-25T01:09:09Z,France,1.0,187.0,1,27156.0,4.0,...,False,NaN,NEW_ENTRY,0.0898,user,0.744,2780.0,11,103.773,"['lilith', 'new wave pop']"
2,54,NaN,2014-07-07,NaN,NaN,0.0,NaN,0,NaN,4.0,...,False,NaN,NaN,0.3700,NaN,0.747,NaN,2,122.030,"['deep groove house', 'house', 'tech house']"
3,0,NaN,2001-01-24,NaN,NaN,1.0,NaN,0,NaN,4.0,...,False,NaN,NaN,0.0865,NaN,0.604,NaN,15,84.099,[]
4,28,NaN,2014-10-03,NaN,NaN,1.0,NaN,0,NaN,4.0,...,False,NaN,NaN,0.0822,NaN,0.761,NaN,26,156.017,"['chill lounge', 'deep chill']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2773057,18,NaN,2024-02-23,NaN,NaN,0.0,NaN,0,NaN,4.0,...,False,NaN,NaN,0.1210,NaN,0.622,NaN,25,122.010,['soulful house']
2773058,65,NaN,2007,NaN,NaN,1.0,NaN,0,NaN,4.0,...,False,NaN,NaN,0.9670,NaN,0.560,NaN,21,110.998,"['british soul', 'new romantic', 'new wave', '..."
2773059,59,NaN,2020-02-14,NaN,Costa Rica,0.0,188.0,0,3213.0,4.0,...,False,NaN,MOVE_UP,0.0921,NaN,0.776,NaN,6,99.877,"['panamanian pop', 'reggaeton', 'urbano latino']"
2773060,21,NaN,2017-10-20,2020-03-01T01:47:43Z,NaN,1.0,NaN,1,NaN,4.0,...,False,NaN,NaN,0.1130,user,0.420,2711.0,16,140.909,[]


In [43]:
X_train.iloc[1]

artist_popularity                                                    51
track_disc_number                                                   NaN
album_release_date                                           1996-04-16
added_at                                           2023-01-25T01:09:09Z
region                                                           France
mode                                                                1.0
rank                                                              187.0
exists                                                                1
streams                                                         27156.0
time_signature                                                      4.0
speechiness                                                      0.0348
track_album_album                                                   NaN
playlist_id                                                       35527
artist_followers                                                

The data has categorical and text features. Catboost, the library used, can deal with them

In [44]:
X_train.drop('track_name', axis=1, inplace=True)

In [45]:
X_train['album_release_date'] = pd.to_datetime(X_train['album_release_date'], format='mixed', errors='coerce')
X_train['album_release_date'] = (X_train['album_release_date'] - X_train['album_release_date'].min()).dt.days

In [46]:
cat_features = ['track_id', 'added_by_type', 'track_artists', 'playlist_id',
                'track_type', 'track_album_album', 'region', 'chart', 
                'trend', 'explicit', 'track_is_local', 'track_track']
text_features = ['name', 'genres', 'available_markets', 'album_name']

In [47]:
set(X_train.columns) - set(cat_features) - set(text_features)

{'acousticness',
 'added_at',
 'added_by_id',
 'album_release_date',
 'album_total_tracks',
 'artist_followers',
 'artist_popularity',
 'danceability',
 'duration_ms',
 'energy',
 'exists',
 'index',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'popularity',
 'rank',
 'speechiness',
 'streams',
 'tempo',
 'time_signature',
 'track_disc_number',
 'track_episode',
 'track_popularity',
 'track_track_number',
 'valence'}

In [48]:
y_train = X_train['exists']  # labels
X_train.drop(['index', 'exists'], inplace=True, axis=1)  # Dropping noise and labels

In [49]:
' '.join(ast.literal_eval(X_train['genres'][0]))

'indie pop la indie pov: indie'

In [50]:
X_train['genres'] = X_train['genres'].apply(lambda x: ' '.join(ast.literal_eval(x)) if pd.notna(x) else '')

In [51]:
X_train.columns

Index(['artist_popularity', 'track_disc_number', 'album_release_date',
       'added_at', 'region', 'mode', 'rank', 'streams', 'time_signature',
       'speechiness', 'track_album_album', 'playlist_id', 'artist_followers',
       'track_popularity', 'album_name', 'valence', 'track_track', 'energy',
       'track_id', 'loudness', 'chart', 'duration_ms', 'acousticness',
       'popularity', 'instrumentalness', 'available_markets', 'track_episode',
       'track_type', 'track_artists', 'key', 'track_track_number', 'name',
       'explicit', 'track_is_local', 'trend', 'liveness', 'added_by_type',
       'danceability', 'added_by_id', 'album_total_tracks', 'tempo', 'genres'],
      dtype='object')

In [52]:
# Filling the gaps so that Catboost can handle the data
X_train[
    X_train[cat_features].isna()
] = ''
X_train[
    X_train[text_features].isna()
] = ''

In [53]:
X_train['genres']

0                              indie pop la indie pov: indie
1                                        lilith new wave pop
2                         deep groove house house tech house
3                                                           
4                                    chill lounge deep chill
                                 ...                        
2773057                                        soulful house
2773058    british soul new romantic new wave new wave po...
2773059               panamanian pop reggaeton urbano latino
2773060                                                     
2773061                        latin christian latin worship
Name: genres, Length: 2773062, dtype: object

In [54]:
train_idx, test_idx = train_test_split(X_train.index, train_size=0.95, random_state=42, shuffle=True)  # For preliminary evaluation

In [55]:
cols = [
    'track_id', 'playlist_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
    'time_signature', 'track_disc_number', 'duration_ms', 'track_episode', 'explicit',
    'track_popularity', 'track_track', 'track_track_number', 'track_type',
    'track_album_album', 'track_artists',
    'rank', 'region', 'chart', 'streams', 'trend', 'genres', 'artist_followers', 'artist_popularity',
    'album_release_date', 'album_total_tracks', 'popularity',
    #'name', 'album_name', 'available_markets',
]  # Columns used for training

In [56]:
dts = pd.to_datetime(X_train['added_at'])
X_train['added_at'] = dts.dt.year
del(dts)

In [57]:
X_train.drop('track_is_local', axis=1, inplace=True)
X_train.drop('added_by_type', axis=1, inplace=True)

In [58]:
X_train

,artist_popularity,track_disc_number,album_release_date,added_at,region,mode,rank,streams,time_signature,speechiness,...,track_track_number,name,explicit,trend,liveness,danceability,added_by_id,album_total_tracks,tempo,genres
0,74,1.0,43438.0,2021.0,,1.0,NaN,NaN,4.0,0.0407,...,2.0,Friday I’m In Love - Recorded at Spotify Studi...,False,,0.0816,0.373,2750.0,2,97.129,indie pop la indie pov: indie
1,51,NaN,35170.0,2023.0,France,1.0,187.0,27156.0,4.0,0.0348,...,NaN,I Love You Always Forever,False,NEW_ENTRY,0.0898,0.744,2780.0,11,103.773,lilith new wave pop
2,54,NaN,41826.0,NaN,,0.0,NaN,NaN,4.0,0.0357,...,NaN,Love Too Deep - Radio Edit,False,,0.3700,0.747,NaN,2,122.030,deep groove house house tech house
3,0,NaN,36914.0,NaN,,1.0,NaN,NaN,4.0,0.0356,...,NaN,No Tiren Las Botellas,False,,0.0865,0.604,NaN,15,84.099,
4,28,NaN,41914.0,NaN,,1.0,NaN,NaN,4.0,0.0613,...,NaN,El Momento de Despertar - Blue Sky Mix,False,,0.0822,0.761,NaN,26,156.017,chill lounge deep chill
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2773057,18,NaN,45344.0,NaN,,0.0,NaN,NaN,4.0,0.0346,...,NaN,Josephine - Eric Kupper Vocal Mix,False,,0.1210,0.622,NaN,25,122.010,soulful house
2773058,65,NaN,39082.0,NaN,,1.0,NaN,NaN,4.0,0.0553,...,NaN,The World & You Tonight - Live at the Royal Al...,False,,0.9670,0.560,NaN,21,110.998,british soul new romantic new wave new wave po...
2773059,59,NaN,43874.0,NaN,Costa Rica,0.0,188.0,3213.0,4.0,0.4300,...,NaN,Party En Mi Casa,False,MOVE_UP,0.0921,0.776,NaN,6,99.877,panamanian pop reggaeton urbano latino
2773060,21,NaN,43027.0,2020.0,,1.0,NaN,NaN,4.0,0.0613,...,NaN,Common Ground,False,,0.1130,0.420,2711.0,16,140.909,


In [59]:
import gc

gc.collect()

30605

In [60]:
set(X_train.columns) - set(cols) # These lead to overfit

{'added_at', 'added_by_id', 'album_name', 'available_markets', 'name'}

<h3> The code below does the training<br>

In [61]:
cat_features_new = list(set(cat_features) & set(cols))
text_features_new = list(set(text_features) & set(cols))

model = catboost.CatBoostClassifier(
    iterations=4500,
    custom_loss='F1',
    learning_rate=0.27,
    random_state=42,
    early_stopping_rounds=20,
    cat_features=cat_features_new,
    text_features=text_features_new,
    train_dir='/kaggle/working'
)

model.fit(
    X_train[cols],
    y_train
)


0:	learn: 0.6486414	total: 8.66s	remaining: 10h 49m 13s
1:	learn: 0.6083441	total: 20.1s	remaining: 12h 33m 56s
2:	learn: 0.5861172	total: 30.2s	remaining: 12h 35m 36s
3:	learn: 0.5536074	total: 40.2s	remaining: 12h 33m 8s
4:	learn: 0.5415962	total: 53.8s	remaining: 13h 26m 21s
5:	learn: 0.5365096	total: 1m 2s	remaining: 13h 5m 37s
6:	learn: 0.5197192	total: 1m 11s	remaining: 12h 49m 43s
7:	learn: 0.5154906	total: 1m 19s	remaining: 12h 25m 15s
8:	learn: 0.5090962	total: 1m 30s	remaining: 12h 33m 28s
9:	learn: 0.5023092	total: 1m 40s	remaining: 12h 31m 55s
10:	learn: 0.5009748	total: 1m 48s	remaining: 12h 18m 10s
11:	learn: 0.4976017	total: 1m 57s	remaining: 12h 9m 32s
12:	learn: 0.4939870	total: 2m 5s	remaining: 12h 16s
13:	learn: 0.4926949	total: 2m 11s	remaining: 11h 44m 50s
14:	learn: 0.4854318	total: 2m 21s	remaining: 11h 46m 59s
15:	learn: 0.4825543	total: 2m 31s	remaining: 11h 45m 52s
16:	learn: 0.4564056	total: 2m 41s	remaining: 11h 47m 50s
17:	learn: 0.4551889	total: 2m 48s	rem

Let's see preliminary scores

In [62]:
y_pred = model.predict(X_train[cols].iloc[test_idx])

recall = recall_score(y_train.iloc[test_idx], y_pred)
precision = precision_score(y_train.iloc[test_idx], y_pred)
print(f'recall: {recall}\nprecision: {precision}')
print(f"Train F1-score: {f1_score(y_train.iloc[test_idx], y_pred)}")

recall: 0.9654870558521325
precision: 0.9453280876773922
Train F1-score: 0.9553012339017706


In [63]:
del(X_train)

<h3>The model is ready to predict the final result</h3>

In [64]:
X_test = pd.read_csv(test_file)

/tmp/ipykernel_18/1435355325.py:1: DtypeWarning: Columns (34,43) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv(test_file)


In [65]:
# Let's save these
'''
os.remove(train_file)
os.remove(test_file)
'''

'\nos.remove(train_file)\nos.remove(test_file)\n'

In [66]:
X_test['genres'] = X_test['genres'].apply(lambda x: ' '.join(ast.literal_eval(x)) if pd.notna(x) else '')
X_test['album_release_date'] = pd.to_datetime(X_test['album_release_date'], format='mixed', errors='coerce')
X_test['album_release_date'] = (X_test['album_release_date'] - X_test['album_release_date'].min()).dt.days
X_test.drop('track_name', axis=1, inplace=True)

In [67]:
X_test[
    X_test[cat_features].isna()
] = ''
X_test[
    X_test[text_features].isna()
] = ''

/tmp/ipykernel_18/3936323533.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_test[


In [68]:
X_test.drop('track_is_local', axis=1, inplace=True)
X_test.drop('added_by_type', axis=1, inplace=True)

In [69]:
test_pred = model.predict(X_test[cols])

submission = pd.DataFrame({"index": X_test["index"],
                           "exists": test_pred})

In [70]:
submission.to_csv("/kaggle/working/submission.csv", index=False)